In [2]:
import polars as pl
import duckdb
import altair as alt
from pathlib import Path
import tomllib

In [3]:
with open("../src/monitor_campista/.streamlit/config.toml", "rb") as f:
    config = tomllib.load(f)

color_scale = config['theme']['colorScale']

In [23]:
con = duckdb.connect(Path('../data/03_gold/monitor_campista_pharma_ads_1880_1884.duckdb'), True)

In [4]:
list(con.sql("SHOW ALL TABLES;").pl()[:,2])

['alinhamento',
 'anuncios',
 'autorizacoes',
 'detalhamento_do_efeito',
 'detalhamento_forma_de_uso',
 'diagramacao',
 'discursos_de_autoridade',
 'doenca_mencionada',
 'duvidas',
 'elementos_de_composicao',
 'extras',
 'filhos',
 'hieraquia_da_informacao',
 'informacoes_indicativas',
 'mencoes_a_lugares',
 'origem',
 'original',
 'pai',
 'palavra_chave_efeito',
 'palavras_chave_produto',
 'primeiras_palavras_do_anuncio',
 'publico_mencionado',
 'responsavel_tecnico',
 'sinal_visual_de_autoridade',
 'substancias',
 'tipificacao_da_imagem_aprox',
 'tipo_de_produto',
 'variacao_tipografica',
 'variacao_typeface',
 'veiculacoes']

In [5]:
con.query("""
select 
    *
from veiculacoes
""").pl()

Identificador,Ano,Edição,Página,Coluna(s) ocupadas,Número de Colunas,Orientação,ano_edicao
str,i64,i64,i64,str,i64,str,str
"""tonico_oriental_cabello""",1880,2,4,"""1""",1,"""↧""","""1880_002"""
"""pilulas_vegeates_assucaradas_b…",1880,6,4,"""5""",1,"""↧""","""1880_006"""
"""novo_depurativo_extracto_salsa…",1880,6,3,"""4,5""",2,"""↧""","""1880_006"""
"""oleo_figado_bacalhau""",1880,8,4,"""5""",1,"""↧""","""1880_008"""
"""novo_depurativo_extracto_salsa…",1880,8,4,"""3,4""",2,"""↧""","""1880_008"""
…,…,…,…,…,…,…,…
"""cura_gonorrhea_sem_massantes_i…",1884,306,4,"""3""",1,"""↧""","""1884_306"""
"""oleo_ducoux_comprido""",1884,306,4,"""3""",1,"""↧""","""1884_306"""
"""importante_descoberta_xarope_s…",1884,306,4,"""3,4""",2,"""↧""","""1884_306"""


# Exploração geral

In [6]:
con.query("""
select
    responsavel_tecnico as Farmacêutico,
    count(distinct Identificador) as Anúncios
from
    responsavel_tecnico
left join
    anuncios using(Identificador)
group by 
    responsavel_tecnico
""").pl()['Farmacêutico'].count()

45

In [7]:
df_ads_by_edition = con.query("""
select 
    ano_edicao,
    count(*) as anuncios,
    min(Página) as menor_pagina,
    max(Página) as maior_pagina,
from veiculacoes
group by 
    ano_edicao
order by 
    ano_edicao
""").pl()

In [8]:
con.query("""
    select
        substancias as Substâncias,
        count(distinct Identificador) as Anúncios
    from
        substancias
    left join
        anuncios using(Identificador)
    group by
        substancias
    order by
        Anúncios desc
    """).pl()

Substâncias,Anúncios
str,i64
"""Ausente""",65
"""Ferro""",16
"""Quina""",8
"""Salsaparrilha""",7
"""Oleo de figado de bacalhau""",7
…,…
"""Louro cereja""",1
"""Extracto de trigo""",1
"""Balsamo de copahiba""",1


In [9]:
df_ads_by_page = con.sql("""
    select
        "Página",
        count(*) as anuncios
    from veiculacoes
    group by
        "Página"
    order by
        Página
    """).pl()

In [10]:
# range_chart = (
#     alt.Chart(df_ads_by_edition)
#     .mark_area()
#     .encode(
#         x=alt.X("ano_edicao:O", title="Edition Year"),
#         y=alt.Y("pagina_primeiro_anuncio:Q", title="Page Number"),
#         y1="pagina_ultimo_anuncio:Q",
#         color=alt.Color("ano:N").scale(range=color_scale),
#     )
# )

# Add line for the first ad page
line_first = (
    alt.Chart(df_ads_by_edition)
    .mark_circle(color="darkblue")
    .encode(x="ano_edicao:O", y="pagina_primeiro_anuncio:Q")
)

# Add line for the last ad page
line_last = (
    alt.Chart(df_ads_by_edition)
    .mark_circle(color=color_scale[0])
    .encode(x="ano_edicao:O", y="pagina_ultimo_anuncio:Q")
)

# Combine all
chart = line_first + line_last

In [11]:
(
    alt.Chart(df_ads_by_page)
    .mark_bar(size=50)
    .encode(
        x=alt.X("Página:O")
        .title("Edição")
        .axis(labelAngle=0)
        .scale(domain=range(1, 9)),
        y=alt.Y("anuncios").title("Contagem Anúncios"),
    )
)

alt.Chart(...)

In [12]:
ad_edition_page = con.query("""
select 
    Ano,ano_edicao,Página
from veiculacoes
""").pl()

In [13]:
# something wrong with this graph
agg = ad_edition_page.group_by(["Página","Ano"]).len()
alt.Chart(agg).mark_bar().encode(
    x=alt.X('Página:O', title='Página'),
    y=alt.Y('count():Q', stack='zero', title='Páginas'),
    color='Ano:N'
)

alt.Chart(...)

In [14]:
(alt.Chart(ad_edition_page).mark_bar()
.transform_aggregate(
    count='count()',
    groupby=['Ano', 'Página']
).encode(
    x='Página:N',
    y=alt.Y('sum(count):Q', title="Veiculações"),
    color=alt.Color('Ano:N').scale(range=color_scale[1::1])
))

alt.Chart(...)

In [15]:
ads_per_edition = (
    alt.Chart(df_ads_by_page)
    .mark_bar(size=70)
    .encode(
        x=alt.X("Página:O")
        .title("Edição")
        .axis(labelAngle=0)
        .scale(domain=range(1, 9)),
        y=alt.Y("anuncios")
        .title("Contagem Anúncios")
        .axis(format=",.0f"),  # Basic formatting without thousands separator
        color=alt.value(color_scale[0]),  # Apply a single color to all bars
    )
)
ads_per_edition

alt.Chart(...)

# Doenças

In [16]:
'doenca_mencionada','tipo_de_produto', 'substancias',

('doenca_mencionada', 'tipo_de_produto', 'substancias')

In [17]:
df_ailments_per_ad = con.sql("""
    select
        doenca_mencionada as Doença,
        count(distinct Identificador) as Anúncios,
        count(distinct ano_edicao) as Veiculações
    from 
        doenca_mencionada
    left join
        veiculacoes using(Identificador)
    group by 
        doenca_mencionada
    order by
        Anúncios desc
""").pl()
df_ailments_per_ad

Doença,Anúncios,Veiculações
str,i64,i64
"""Rheumatismo""",27,327
"""Ausente""",26,169
"""Anemia""",23,264
"""Chlorose [chlorosis]""",20,243
"""Escrofula (escrophulasa scroph…",17,239
…,…,…
"""Inflamação dos pulmões""",1,117
"""Descaimento das energias""",1,6
"""Inflamação intestinal""",1,7


In [18]:
(
    alt.Chart(df_ailments_per_ad)
        .mark_bar()
        .encode(
        x=alt.X("Doença", sort="-y"),
        y=alt.Y("Anúncios"),
        color=alt.Color("Veiculações")
    )
)

alt.Chart(...)

In [19]:
long = (
    df_ailments_per_ad
    .unpivot(
        index='Doença',                # columns to keep as-is
        on=['Anúncios', 'Veiculações'], # columns to stack
        variable_name='Métrica',        # name for the “former column name” column
        value_name='Valor'              # name for the stacked values
    )
)

alt.Chart(long).mark_bar().encode(
    x=alt.X('Doença:N', sort='-y'),
    y='Valor:Q',
    color='Métrica:N'
)

alt.Chart(...)

In [20]:
alt.Chart(df_ailments_per_ad).mark_circle(size=200).encode(
    x=alt.X('Anúncios:Q', title='Anúncios'),
    y=alt.Y('Veiculações:Q', title='Veiculações'),
    tooltip=['Doença','Anúncios','Veiculações'],
    color=alt.Color(
        'Doença',
        scale=alt.Scale(range=color_scale)   
    ),
)

alt.Chart(...)

In [21]:
base = alt.Chart(df_ailments_per_ad).encode(
    x=alt.X('Anúncios:Q', title='Anúncios'),
    y=alt.Y('Veiculações:Q', title='Veiculações')
)

points = base.mark_circle(size=200)
labels = base.mark_text(
    align='center',
    baseline='bottom',
    dy=-8,               # nudge text up a little
    fontSize=9
).encode(text='Doença:N')

points + labels

alt.LayerChart(...)

In [22]:
base = alt.Chart(df_ailments_per_ad).encode(
    x=alt.X("Doença:N",sort=alt.SortField(field="Anúncios", order='descending')) 
)

bar1 = base.mark_bar(color=color_scale[1]).encode(
    y=alt.Y("Anúncios:Q").axis(titleColor=color_scale[0]) 
)

bar2 = base.mark_bar(color=color_scale[3]).encode(
    y=alt.Y("Veiculações:Q").axis(titleColor=color_scale[1])
)

# 4. Layer them and create independent y-axes
final_chart = alt.layer(bar1, bar2).resolve_scale(y="independent").configure_mark(
        opacity=0.6
    )

final_chart

alt.LayerChart(...)

In [23]:
df_ailments_count_per_ad = con.sql("""
    select
        Identificador as anuncio,
        count(distinct 
            case when doenca_mencionada = 'Ausente' then null
            else doenca_mencionada
            end
        ) as doencas
    from
        anuncios
    left join
        doenca_mencionada using(Identificador)
    group by
        Identificador
""").pl()
df_ailments_count_per_ad[:,1].unique()

doencas
i64
0
1
2
3
4
…
13
16
18


In [24]:
(
    alt.Chart(df_ailments_count_per_ad)
    .mark_bar()
    .encode(
        x=alt.X('doencas:N', title='Contagem doenças mencionadas', sort='x')
        .axis(labelAngle=0),
        y=alt.Y('count()', title='Anúncios'),
        color=alt.Color(value=color_scale[0])
    )
)

alt.Chart(...)

In [25]:
(
    alt.Chart(df_ailments_count_per_ad)
    .mark_bar()
    .encode(
        x=alt.X('doencas:N', title='Contagem doenças mencionadas', sort='x')
            .axis(labelAngle=0),
        y=alt.Y('count()', title='Anúncios'),
        # color=alt.Color("Ano:N").scale(range=["grey"] + color_scale)
    )
         .properties(title="Doenças por Anúncio")
)

alt.Chart(...)

# Análise Gráfica

In [26]:
'informacoes_indicativas', 
'detalhamento_do_efeito', 
'detalhamento_forma_de_uso', 

'variacao_typeface', 
'variacao_tipografica', 
'alinhamento', 
'diagramacao', 

'hieraquia_da_informacao', 
'elementos_de_composicao',  

'tipificacao_da_imagem_aprox', 
'primeiras_palavras_do_anuncio', 
'sinal_visual_de_autoridade', 

'original';


In [27]:
property = "informacoes_indicativas"
def df(property):
    return con.sql(f"""
    select
        {property},
        count(distinct Identificador) as Anúncios
    from
        anuncios
    left join
        {property} using(Identificador)
    group by
        {property}
    order by
        Anúncios desc
    """).pl()
df(property)

informacoes_indicativas,Anúncios
str,i64
"""Depósito""",77
"""Responsável técnico/preparado …",61
"""Comercialização""",50
"""Comercialização/Depósito (dúbi…",21
"""Preço""",17
"""Produção""",11
"""Indefinido""",2
"""Segredo""",1
"""[rua] se dirá [ ]""",1


In [28]:
def hist(property, invert_axis=False):  
    x = alt.X(property, title="", sort="-x" if invert_axis else "-y")
    y = alt.Y("Anúncios", title="Anúncios")
    if invert_axis:
        x,y = y,x     
    return (
        (
            alt.Chart(df(property))
            .mark_bar()
            .encode(
                x=x,
                y=y,
                color=alt.Color(value=color_scale[0]),
            )
            .properties(title=property)
        )
    )
hist(property, True)

alt.Chart(...)

In [74]:
x = con.sql("""
WITH total_edicoes AS (
    SELECT COUNT(DISTINCT ano_edicao) AS total
    FROM veiculacoes
)
SELECT 
    MAX(Página) AS quantidade_de_paginas, 
    COUNT(DISTINCT ano_edicao) AS edicoes,
    Ano,
    te.total
FROM veiculacoes
JOIN total_edicoes te ON 1 = 1
GROUP BY Ano, ano_edicao, te.total


""")
x
chart = alt.Chart(x).mark_bar().encode(
    x=alt.X("quantidade_de_paginas:O", title="Quantidade de Páginas"),
    y=alt.Y("edicoes:Q", title="Número de Edições"),
    color=alt.Color("Ano:N", legend=alt.Legend(title="Ano")),
    text=alt.Text(
        "edicoes:Q", 
        format=".1f"  # Formatar como percentual com uma casa decimal
    )
).transform_calculate(
    percentual='datum.edicoes / datum.total * 100'
).encode(
    text=alt.Text("percentual:Q", format=".1f")  # Exibindo o percentual sobre as barras
).properties(
    title="Histograma de Edições por Quantidade de Páginas"
)
chart

# alt.Chart(x).mark_bar().encode(
#     x=alt.X("quantidade_de_paginas:O", title="Quantidade de Páginas"),  # O eixo X é a quantidade de páginas
#     y=alt.Y("edicoes:Q", title="Número de Edições"),  # O eixo Y é o número de edições
#     color=alt.Color("Ano:N", legend=alt.Legend(title="Ano"))  # A cor é o Ano
# ).properties(
#     title="Histograma de Edições por Quantidade de Páginas"
# )

alt.Chart(...)

In [76]:
x.count_uniqu()

AttributeError: This relation does not contain a column by the name of 'count_uniqu'

In [73]:
chart = alt.Chart(x).mark_bar().encode(
    x=alt.X("quantidade_de_paginas:O", title="Quantidade de Páginas"),
    y=alt.Y("edicoes:Q", title="Número de Edições"),
    color=alt.Color("Ano:N", legend=alt.Legend(title="Ano")),
    text=alt.Text(
        "edicoes:Q", 
        format=".1f"  # Formatar como percentual com uma casa decimal
    )
).transform_calculate(
    percentual='datum.edicoes / datum.total * 100'
).encode(
    text=alt.Text("percentual:Q", format=".1f")  # Exibindo o percentual sobre as barras
).properties(
    title="Histograma de Edições por Quantidade de Páginas"
)
chart

alt.Chart(...)

# Análise Discursiva

In [29]:
'origem','mencoes_a_lugares',
'palavra_chave_efeito', 
'palavras_chave_produto', 
'discursos_de_autoridade', 
'publico_mencionado';

In [30]:
 'extras','autorizacoes'

('extras', 'autorizacoes')

In [31]:
df = con.sql("""
select *
from anuncios
""").pl()
df

Identificador,ID,Link,Produto ofertado (título completo),Preço,Comercialização,Depósito,Produção,Observações,Quantidade de variações tipográficas (aprox.),Original (primeira aparição),Derivados,Status,Dúvidas,image_url
str,i64,str,str,str,str,str,str,str,i64,str,str,str,str,str
"""oleo_figado_bacalhau""",1,"""https://drive.google.com/file/…","""Oleo puro de figado de bacalha…","""-""","""Vende-se em todas as pharmacia…","""-""","""-""",null,5,null,null,"""Finalizado""",null,"""https://drive.google.com/thumb…"
"""tonico_oriental_cabello""",2,"""https://drive.google.com/file/…","""O tonico oriental para o cabel…","""-""","""Vende-se em todas as pharmacia…","""-""","""-""",null,4,null,null,"""Finalizado""",null,"""https://drive.google.com/thumb…"
"""pilulas_vegeates_assucaradas_b…",3,"""https://drive.google.com/file/…","""Pilulas vegetaes assucaradas d…","""-""","""Vende-se em todas as farmácias…","""-""","""-""",null,4,null,null,"""Finalizado""",null,"""https://drive.google.com/thumb…"
"""og_novo_depurativo_extracto_sa…",4,"""https://drive.google.com/file/…","""Novo depurativo extracto fluid…","""Preço de um vidro 2$500 Preço …","""-""","""DEPOSITO:—Pharmacia de Vicente…","""-""","""N. B. — Todos os rotulos e pro…",7,null,"""novo_depurativo_extracto_salsa…","""Finalizado""",null,"""https://drive.google.com/thumb…"
"""salsaparrilha_bristol""",5,"""https://drive.google.com/file/…","""Salsaparrilha de bristol""","""-""","""Vende-se em todas as pharmacia…","""-""","""-""",null,2,null,null,"""Finalizado""",null,"""https://drive.google.com/thumb…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""o_maior_successo_epocha""",146,"""https://drive.google.com/file/…","""Pectrolectrina""","""Preços sem competencia""","""-""","""Deposito na cidade de Campos d…","""-""","""-""",8,null,null,"""Finalizado""",null,"""https://drive.google.com/thumb…"
"""licor_coca""",147,"""https://drive.google.com/file/…","""Licor de coca""","""-""","""Vende-se em casa de Julião Pes…","""-""","""-""","""-""",3,null,null,"""Finalizado""",null,"""https://drive.google.com/thumb…"
"""pilulas_vermifugas_dr_oliveira""",148,"""https://drive.google.com/file/…","""Pilulas vermifugas do Dr Olive…","""Caixa 2$000""","""Pharmacia Ezequiel 70 - Rua do…","""-""","""-""","""preparado pharmaceutico, formu…",7,null,null,"""Finalizado""",null,"""https://drive.google.com/thumb…"


In [32]:
df["Identificador"].unique().count()

149

In [96]:
df = con.sql("""
with quantidade_paginas as (
select
    max(Página) as total_paginas
from
    veiculacoes
group by 
    ano_edicao
)
select 
    total_paginas,
    count(*) as edicoes
from
    quantidade_paginas
group by 
    total_paginas
""").pl()
df

total_paginas,edicoes
i64,i64
3,41
6,6
5,2
4,681
7,1
8,2


In [124]:
(
    alt.Chart(
        con.sql("""
        with quantidade_paginas as (
        select
            max(Página) as total_paginas
        from
            veiculacoes
        group by 
            ano_edicao
        )
        select 
            total_paginas,
            count(*) as edicoes
        from
            quantidade_paginas
        group by 
            total_paginas
        """).pl()     
    )
    .mark_bar()
    .encode(
        x=alt.X("total_paginas:N", axis=alt.Axis(labelAngle=0)),
        y="edicoes",
    )
)

alt.Chart(...)

In [140]:
df = (
    con.sql("""
    with quantidade_paginas as (
        select
            max(Página) as total_paginas
        from
            veiculacoes
        group by
            ano_edicao
    )
    select
        total_paginas,
        count(*) as edicoes
    from
        quantidade_paginas
    group by
        total_paginas
""")
    .pl()
    .to_pandas()
)

def hist_with_percent(df,x_col,y_col):
    df["percent"] = df[y_col] / df[y_col].sum()
    
    bars = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(f"{x_col}:N", axis=alt.Axis(labelAngle=0)),
            y=alt.X(f"{y_col}"),
            color=alt.Color(value=color_scale[0])
        )
    )
    
    text = (
        alt.Chart(df)
        .mark_text(dy=-6)
        .encode(
            x=f"{x_col}:N", y=f"{y_col}:Q", text=alt.Text("percent:Q", format=".0%")
        )
    )
    return bars + text

hist_with_percent(df,"total_paginas","edicoes")


alt.LayerChart(...)

In [163]:
df = (
    con.sql("""
    select
        Identificador as anuncio,
        count(distinct
            case when doenca_mencionada = 'Ausente' then null
            else doenca_mencionada
            end
        ) as doencas
    from
        anuncios
    left join
        doenca_mencionada using(Identificador)
    group by
        Identificador
""")
    .pl()
    .to_pandas()
)

def hist_with_percent(df,x_col,y_col):
    df["percent"] = df[y_col] / df[y_col].count()
    
    bars = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(f"{x_col}:N", axis=alt.Axis(labelAngle=0)),
            y=alt.X(f"{y_col}"),
            color=alt.Color(value=color_scale[0])
        )
    )
    
    text = (
        alt.Chart(df)
        .mark_text(dy=-6)
        .encode(
            x=f"{x_col}:N", y=f"{y_col}:Q", text=alt.Text("percent:Q", format=".0%")
        )
    )
    return bars + text

hist_with_percent(df,"anuncio","doencas")


alt.LayerChart(...)

In [173]:
df

,anuncio,doencas,percent
0,licor_centaurea_2,6,0.039474
1,pastilhas_peitoraes,9,0.059211
2,xarope_massa,8,0.052632
3,escrophulas,1,0.006579
4,"a_celebre_moussenade_aviso_1,2,3,4_4",10,0.065789
...,...,...,...
147,"a_celebre_moussenade_aviso_1,2,3,4_illmjosejoa...",0,0.000000
148,casa_ingleza_vaselina_camphorada,1,0.006579
149,o_melhor_remedio_febres_intermittentes,1,0.006579
150,xarope_tres_quinas,0,0.000000


In [179]:
x_col = "doencas"
x_title = "Contagem doenças"
y_title = "Anúncios"

def hist_perc(df, x_col, x_title, y_title):
    bars = (alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X(f"{x_col}:N", title=x_title, sort="x").axis(
            labelAngle=0
        ),
        y=alt.Y("count()", title=y_title),
        color=alt.Color(value=color_scale[0]),
    ))

    text = (
        alt.Chart(df)
        .transform_aggregate(
            count="count()", groupby=[x_col]
        )
        .transform_window(
            total="sum(count)"
        )
        .transform_calculate(
            percent="round(datum.count / datum.total * 100, 1) + '%'"
        )
        .mark_text(dy=-6, fontSize=11)
        .encode(
            x=alt.X(f"{x_col}:N", sort="x"),
            y="count:Q",
            text="percent:N"
        )
    )

    return bars + text
    
hist_perc(df, x_col, x_title, y_title)

alt.LayerChart(...)

In [183]:

def hist_perc(df, x_col, x_title, y_title):
    # Step 1: aggregate counts per category
    base = (
        alt.Chart(df)
        .transform_aggregate(
            count="count()", groupby=[x_col]
        )
        # Step 2: compute grand total (no groupby → one row)
        .transform_joinaggregate(
            total="sum(count)"
        )
        # Step 3: calculate percent using that grand total
        .transform_calculate(
            percent="round(datum.count / datum.total * 100, 1) + '%'"
        )
    )

    bars = (
        base.mark_bar()
        .encode(
            x=alt.X(f"{x_col}:N", title=x_title, sort="x", axis=alt.Axis(labelAngle=0)),
            y=alt.Y("count:Q", title=y_title),
            color=alt.Color(value=color_scale[0]),
        )
    )

    text = (
        base.mark_text(dy=-6, fontSize=11)
        .encode(
            x=alt.X(f"{x_col}:N", sort="x"),
            y="count:Q",
            text="percent:N"
        )
    )

    return bars + text
hist_perc(df, x_col, x_title, y_title)

alt.LayerChart(...)

In [186]:
(    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("doencas:N", title="Contagem doenças mencionadas", sort="x").axis(
            labelAngle=0
        ),
        y=alt.Y("count()", title="Anúncios"),
        color=alt.Color(value=color_scale[0]),
    ))

alt.Chart(...)

In [97]:
df["edicoes"].sum()

733

In [98]:
681/733

0.9290586630286494

In [195]:
con.sql("""
select count(distinct palavras_chave_produto)
from palavras_chave_produto

""")

┌────────────────────────────────────────┐
│ count(DISTINCT palavras_chave_produto) │
│                 int64                  │
├────────────────────────────────────────┤
│                                    139 │
└────────────────────────────────────────┘

In [250]:
df = con.sql("""
with autorizacoes as (
select 
    case 
        when autorizacoes = 'Ausente' then 'Ausente'
        when autorizacoes = 'Governo Imperial' then 'Governo Imperial'
        when autorizacoes = 'Pharmacopéa official da França' then 'Pharmacopéa official da França'
        when autorizacoes = 'Academia de Medicina de Paris' then 'Academia de Medicina de Paris'
        else 'Exma. Junta Central de Hygiene'
    end as Autoridade,
    count(distinct Identificador) as Anúncios,
    case 
        when autorizacoes = 'Ausente' then 'Ausente'
        else 'Presente'
    end as Autorização
from anuncios
left join 
    autorizacoes using(Identificador)
group by autorizacoes
)
select 
    Autoridade, 
    Autorização, 
    sum(Anúncios)::int64 as Anúncios
from autorizacoes
group by Autoridade, Autorização
""").pl()

x_col = "Autorização"
y_col =  "Anúncios"
color_col = "Autoridade"
(
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(f"{x_col}:N", axis=alt.Axis(labelAngle=0)),
            y=alt.Y(f"{y_col}"),
            color=alt.Color(color_col, scale=alt.Scale(range=color_scale))
        )
    )

alt.Chart(...)

In [57]:
# analyze later
# def df_to_hist_new(df, x_col, y_col, color_col=None):
#     df = df.to_pandas()

#     df["percent"] = df.groupby(x_col)[y_col].transform(lambda x: x / x.sum())

#     if color_col:
#         color = alt.Color(
#             color_col,
#         ).scale(range=color_scale)
#     else:
#         color = alt.value(color_scale[0])

#     bars = (
#         alt.Chart(df)
#         .mark_bar()
#         .encode(
#             x=alt.X(f"{x_col}:N", axis=alt.Axis(labelAngle=0, title="Autorização")),
#             y=alt.Y(f"{y_col}:Q", title="Anúncios", stack="zero"),
#             color=color,
#         )
#     )

#     totals = (
#         df.groupby(x_col)[y_col]
#         .sum()
#         .reset_index()
#         .assign(percent=lambda d: d[y_col] / d[y_col].sum())
#     )

#     text = (
#         alt.Chart(totals)
#         .mark_text(dy=-10, fontWeight="bold")
#         .encode(
#             x=f"{x_col}:N",
#             y=alt.Y(f"{y_col}:Q", stack="zero"),
#             text=alt.Text("percent:Q", format=".0%"),
#         )
#     )

#     return bars + text


In [ ]:
# test later 
    # df = con.sql("""
    # with autorizacoes as (
    # select
    #     case
    #         when autorizacoes = 'Ausente' then 'Ausente'
    #         when autorizacoes = 'Governo Imperial' then 'Governo Imperial'
    #         when autorizacoes = 'Pharmacopéa official da França' then 'Pharmacopéa official da França'
    #         when autorizacoes = 'Academia de Medicina de Paris' then 'Academia de Medicina de Paris'
    #         else 'Exma. Junta Central de Hygiene'
    #     end as Autoridade,
    #     count(distinct Identificador) as Anúncios,
    #     case
    #         when autorizacoes = 'Ausente' then 'Ausente'
    #         else 'Presente'
    #     end as Autorização
    # from anuncios
    # left join
    #     autorizacoes using(Identificador)
    # group by autorizacoes
    # )
    # select Autoridade, Autorização, sum(Anúncios)::int64 as Anúncios
    # from autorizacoes
    # group by Autoridade, Autorização
    # """).pl()

    # _ = st.altair_chart(df_to_hist_new(df, "Autorização", "Anúncios", "Autoridade"))


In [240]:
df = con.sql("""
with autorizacoes as (
select 
    case 
        when autorizacoes = 'Ausente' then 'Ausente'
        when autorizacoes = 'Governo Imperial' then 'Governo Imperial'
        when autorizacoes = 'Pharmacopéa official da França' then 'Pharmacopéa official da França'
        when autorizacoes = 'Academia de Medicina de Paris' then 'Academia de Medicina de Paris'
        else 'Exma. Junta Central de Hygiene'
    end as Autoridade,
    count(distinct Identificador) as Anúncios,
    case 
        when autorizacoes = 'Ausente' then 'Ausente'
        else 'Presente'
    end as Autorização
from anuncios
left join 
    autorizacoes using(Identificador)
group by autorizacoes
)
select Autoridade, Autorização, sum(Anúncios) as Anúncios
from autorizacoes
group by Autoridade, Autorização

""")
df.pl()

Autoridade,Autorização,Anúncios
str,str,"decimal[38,0]"
"""Pharmacopéa official da França""","""Presente""",3
"""Academia de Medicina de Paris""","""Presente""",3
"""Ausente""","""Ausente""",112
"""Exma. Junta Central de Hygiene""","""Presente""",38
"""Governo Imperial""","""Presente""",3


In [268]:
df = con.sql("""
select Identificador, count(*) as n
from autorizacoes
group by Identificador
having n > 1
""").pl()
df

Identificador,n
str,i64
"""o_grande_purificador_reconstit…",2
"""pastilhas_peitoraes""",2
"""ferro_girard""",2
"""o_grande_purificador_RECONSTIT…",2
"""ferro_leras""",2
"""vermouthina_substitutiva_agua_…",2
"""xarope_quina_ferro_grimault""",2


In [270]:
df = con.sql("""
select * 
from autorizacoes
where Identificador = 'ferro_girard'
""").pl()
df

# muito curioso de se pensar nas implicações de contabilizar o percentual das autorizações

Identificador,autorizacoes
str,str
"""ferro_girard""","""Academia de Medicina de Paris"""
"""ferro_girard""","""Exma. Junta de Hygiene do Braz…"


In [281]:
df = con.sql("""
select count(distinct Identificador)
from mencoes_a_lugares
where mencoes_a_lugares <> 'Nenhum'
""").pl()
df

count(DISTINCT Identificador)
i64
117


In [299]:
df = con.sql("""
with presenca_imagem as (
select 
    case 
        when tipificacao_da_imagem_aprox = 'Ausente' then 'Ausente'
        else 'Presente'
    end as 'Presença de Imagem',
    Identificador
from tipificacao_da_imagem_aprox
)
select
    "Presença de Imagem", count(distinct Identificador)
from presenca_imagem
group by "Presença de Imagem"
""").pl()
df

Presença de Imagem,count(DISTINCT Identificador)
str,i64
"""Presente""",35
"""Ausente""",118


In [301]:
df = con.sql("""
select 
    Identificador, count(distinct variacao_tipografica)
from 
    variacao_tipografica
group by 
    Identificador
""").pl()
df

Identificador,count(DISTINCT variacao_tipografica)
str,i64
"""vinho_peptona_pepsica_chapotea…",4
"""a_celebre_moussenade_aviso_1,2…",4
"""a_celebre_moussenade_2_colunas""",5
"""xarope_zed_codeina_tolu""",6
"""rob_doliaria_ferro_composto_02""",4
…,…
"""muita_attencao_pomada_indiana""",4
"""gonorrheas_novo""",2
"""doenca_do_estomago_elexir_camo…",1


In [15]:
df = con.sql("""
select 
    "Quantidade de variações tipográficas (aprox.)" as 'Quantidade variações tipográficas',
     count(distinct Identificador), 
from 
    anuncios
group by 
    "Quantidade de variações tipográficas (aprox.)"
""").pl()
df

Quantidade variações tipográficas,count(DISTINCT Identificador)
i64,i64
5,13
8,21
9,13
14,1
6,16
…,…
11,5
7,13
4,14


In [34]:
df = con.sql("""
select 
    palavras_chave_produto as 'Palavra-chave produto', 
    count(distinct Identificador) as Anúncios,
    count(distinct ano_edicao || Identificador) as Veiculações
from 
    palavras_chave_produto
left join 
    veiculacoes
        using(Identificador)
group by
    palavras_chave_produto
order by
    Veiculações desc
""").pl()
df

Palavra-chave produto,Anúncios,Veiculações
str,i64,i64
"""Eficaz (efficacissimo)""",18,618
"""Ausente""",28,391
"""Agradável""",21,342
"""Poderoso(a)""",16,275
"""Grande""",4,249
…,…,…
"""Prodigioso""",1,1
"""Afamada(o)""",1,1
"""Valiosissimo""",1,1


In [47]:
df = con.sql("""
select 
    palavras_chave_produto as 'Palavra-chave produto', 
    count(*) as Veiculações
from 
    veiculacoes
left join 
    palavras_chave_produto
        using(Identificador)
where 
    palavras_chave_produto is not null
group by
    palavras_chave_produto
order by
    Veiculações desc
""").pl()
df

Palavra-chave produto,Veiculações
str,i64
"""Eficaz (efficacissimo)""",619
"""Ausente""",393
"""Agradável""",342
"""Poderoso(a)""",276
"""Grande""",249
…,…
"""Prodigioso""",1
"""Afamada(o)""",1
"""Aperiente""",1


In [56]:
df = con.sql("""
select 
    count(distinct Identificador),
from 
    anuncios
where
    "Original (primeira aparição)" is null
""").pl()
df

count(DISTINCT Identificador)
i64
123


In [35]:
# con.close() 